In [1]:
# Import dependencies
import pandas as pd
import sqlalchemy
from pprint import pprint
from flask import Flask, jsonify
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine, func

In [2]:
# Create engine to wardata.sqlite
engine = create_engine("sqlite:///data/wardata.sqlite")
DBSession = sessionmaker(bind=engine)
session = DBSession()

In [3]:
Base = declarative_base()
class Baseball(Base):
    __tablename__ = "baseball"
    player_id = Column(Integer, primary_key=True)
    rank_in_country = Column(Integer)
    player_name = Column(String)
    war = Column(Float)
    year_min = Column(Integer)
    year_max = Column(Integer)
    age_range = Column(String)
    birth_location = Column(String)
    country = Column(String)
    position = Column(String)

In [4]:
variable = session.query(Baseball)

#Sample Query
new_df = pd.read_sql("SELECT * FROM baseball", engine)
new_df

,player_id,rank_in_country,player_name,war,year_min,year_max,age_range,birth_location,country,position
0,0,1,Barry Bonds,162.8,1986,2007,21-42,"Riverside, CA",USA,Hitter
1,1,2,Babe Ruth,162.7,1914,1935,19-40,"Baltimore, MD",USA,Hitter
2,2,3,Willie Mays,156.1,1948,1973,17-42,"Westfield, AL",USA,Hitter
3,3,4,Ty Cobb,151.4,1905,1928,18-41,"Narrows, GA",USA,Hitter
4,4,5,Henry Aaron,143.0,1954,1976,20-42,"Mobile, AL",USA,Hitter
...,...,...,...,...,...,...,...,...,...,...
2575,2747,87,Héctor Fajardo,-1.1,1991,1995,20-24,"Sahuayo, MEX",MEX,Pitcher
2576,2749,89,José Peña,-1.2,1969,1972,26-29,"Ciudad Juarez, MEX",MEX,Pitcher
2577,2751,91,Luis Mendoza,-1.2,2007,2013,23-29,"Veracruz, MEX",MEX,Pitcher
2578,2752,92,Edgar González,-1.3,2003,2013,20-30,"San Nicolas de los Garza, MEX",MEX,Pitcher


In [ ]:
#Flask Dictionaries

#Baseball Dictionary
baseball_dict = new_df.to_dict("records")

In [ ]:
# Flask Setup
app = Flask(__name__)

# Flask Routes
@app.route("/")
def homepage():
    return (
        f"Welcome to the homepage!"
        "<br>"
        f"Available Routes:"
        "<br>"
        f"/api/v1.0/baseball"
        "<br>"
        "Use this route to search for a specific country by country code (i.e. USA, MEX, CAN)"
        "/api/v1.0/country/<country>"
        "<br>"
        "Use this route to search for top ten players by country code (i.e. USA, MEX, CAN)"
        "/api/v1.0/topten/<country>"
        "<br>"
        "Use this route to search for players by position (Hitter or Pitcher)"
        "/api/v1.0/position/<position>"
    )

@app.route("/api/v1.0/baseball")
def baseball():
    """Return baseball data as json"""
    return jsonify(baseball_dict)

@app.route("/api/v1.0/country/<country>")
def country(country):
    """Return players without missing data as json by country"""
    temp_df = pd.read_sql(f"SELECT * FROM baseball WHERE country LIKE '{country.upper()}';", engine)
    if temp_df.empty == False:
        return jsonify(temp_df.to_dict("records"))
    else:
        return jsonify({"error": "Country not found or doesn't contain records."})
    
@app.route("/api/v1.0/position/<position>")
def position(position):
    """Return players without missing data as json by position"""
    temp_df = pd.read_sql(f"SELECT * FROM baseball WHERE position LIKE '{position.capitalize()}';", engine)
    if temp_df.empty == False:
        return jsonify(temp_df.to_dict("records"))
    else:
        return jsonify({"error": "Country not found or doesn't contain records."})

@app.route("/api/v1.0/topten/<country>")
def top_ten_by_country(country):
    """Return top ten players without missing data ordered by war as json by country"""
    temp_df = pd.read_sql(f"SELECT * FROM baseball WHERE country LIKE '{country.upper()}' ORDER BY war LIMIT 10;", engine)
    if temp_df.empty == False:
        return jsonify(temp_df.to_dict("records"))
    else:
        return jsonify({"error": "Country not found or doesn't contain records."})
    
@app.route("/api/v1.0/country/<country>")
def country(country):
    """Return players without missing data as json by country"""
    temp_df = pd.read_sql(f"SELECT * FROM baseball WHERE country LIKE '{country.upper()}';", engine)
    if temp_df.empty == False:
        return jsonify(temp_df.to_dict("records"))
    else:
        return jsonify({"error": "Country not found or doesn't contain records."})

@app.route("/api/v1.0/country/<country>")
def country(country):
    """Return players without missing data if they played before a certain year"""
    temp_df = pd.read_sql(f"SELECT * FROM baseball WHERE country LIKE '{country.upper()}';", engine)
    if temp_df.empty == False:
        return jsonify(temp_df.to_dict("records"))
    else:
        return jsonify({"error": "Country not found or doesn't contain records."})

if __name__ == "__main__":
    app.run(debug=True, use_reloader = False)